In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Reading data
import numpy as np
import pandas as pd
data= pd.read_csv('../input/multijet-primary-dataset/MultiJetRun2010B.csv')
data.head()

In [ ]:
#No null or categorical data
data.info()

In [ ]:
data.describe()

In [ ]:
#importing libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [ ]:
#seaborn to plot between any two columns, nJet which is the target is highlighted
sns.pairplot(
    data,hue="nJets",
    x_vars=["MR", "HT", "E1","E2","Rsq","MET"],
    y_vars=["MR", "HT", "E1","E2","Rsq","MET",],
)

In [ ]:
#Correlation matrix, Lumi is directly correlated to even can be ignored
sns.set_style()
plt.figure(figsize=(25,15))
sns.heatmap(data.corr(),annot=True)
plt.show()

In [ ]:
#model building using xgboost regressor
import xgboost as xg

In [ ]:
#split data, tagert is nJets
#save run column for the final tabel
#several columns dropped based on feature engineering to improve accuracy
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=['nJets','Event','Lumi','nBJets','Pz2','MET','Pz1']),
    data['nJets'],
    test_size=0.25,
    random_state=42,
    stratify=data['nJets']
)
run = X_test['Run'].copy()
X_test = X_test.drop('Run',axis=1)
X_train = X_train.drop('Run',axis=1)

In [ ]:
X_train.head(2)

In [ ]:
xgb_r = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 42)

In [ ]:
xgb_r.fit(X_train, y_train)

In [ ]:
# cheking training accuracy
y_pred = xgb_r.predict(X_train)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_train,predictions)
accuracy

In [ ]:
# cheking initial test accuracy
y_pred = xgb_r.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test,predictions)
accuracy
#grid search CV was run but the accuracy didn't improve

In [ ]:
feature_importance = np.array(xgb_r.feature_importances_)
feature_names = np.array(X_train.columns)
data1={'feature_names':feature_names,'feature_importance':feature_importance}
data_plt = pd.DataFrame(data1)
sns.barplot(x=data_plt['feature_importance'], y=data_plt['feature_names'])
plt.xlabel('feature_importance')
plt.ylabel('feature_names')
plt.show()

In [ ]:
table = pd.DataFrame({'Run':run,'nJets':predictions})
table

In [ ]:
table.to_csv('CERN proton collision solution.csv',index=False)